In [ ]:
import pandas as pd
nb_hrs_studies = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
avg_grade = [5.5, 5.8, 6.8, 7.2, 7.4, 7.8, 8.2, 8.8, 9.3, 9.4]
data = pd.DataFrame({'nb_hrs_studies': nb_hrs_studies, 'avg_grade': avg_grade})
data


In [ ]:
import matplotlib.pyplot as plt
plt.scatter(data['nb_hrs_studies'], data['avg_grade'])
plt.xlabel('nb_hrs_studies')
plt.ylabel('avg_grades')


In [ ]:
from sklearn.datasets import make_regression
X,y = make_regression(n_samples=1000,n_features=5,n_informative=5,noise=100)


In [ ]:
pd.DataFrame(X).describe()

In [ ]:
pd.Series(y).hist()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [ ]:
from river.linear_model import LinearRegression
model = LinearRegression()


In [ ]:
# fit the model
for x_i,y_i in zip(X_train,y_train):
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    model.learn_one(x_json,y_i)


In [ ]:
# predict on the test set
import pandas as pd
preds = []
for x_i in X_test:
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    preds.append(model.predict_one(x_json))


In [ ]:
# compute accuracy
from sklearn.metrics import r2_score
r2_score(y_test, preds)


In [ ]:
from river.tree import HoeffdingAdaptiveTreeRegressor
model = HoeffdingAdaptiveTreeRegressor(seed=42)

# fit the model
for x_i,y_i in zip(X_train,y_train):
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    model.learn_one(x_json,y_i)

# predict on the test set
import pandas as pd
preds = []
for x_i in X_test:
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    preds.append(model.predict_one(x_json))

# compute accuracy
from sklearn.metrics import r2_score
r2_score(y_test, preds)


In [ ]:
def evaluate_HATR(grace_period, leaf_prediction, model_selector_decay):

    # model pipeline
    model = (
        HoeffdingAdaptiveTreeRegressor(
            grace_period=grace_period,
            leaf_prediction=leaf_prediction,
            model_selector_decay=model_selector_decay,
            seed=42)
    )

    # fit the model
    for x_i,y_i in zip(X_train,y_train):
        x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
        model.learn_one(x_json,y_i)

    # predict on the test set
    preds = []
    for x_i in X_test:
        x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
        preds.append(model.predict_one(x_json))

    # compute accuracy
    return r2_score(y_test, preds)


In [ ]:
grace_periods=[0,5,10,]
leaf_predictions=['mean','adaptive']
model_selector_decays=[ 0.3, 0.8,  0.95]


In [ ]:
results = []
i = 0
for grace_period in grace_periods:
    for leaf_prediction in leaf_predictions:
        for model_selector_decay in model_selector_decays:
            print(i)
            i = i+1
            results.append([grace_period, leaf_prediction, model_selector_decay,evaluate_HATR(grace_period, leaf_prediction, model_selector_decay)])


In [ ]:
pd.DataFrame(results, columns=['grace_period', 'leaf_prediction', 'model_selector_decay', 'r2_score' ]).sort_values('r2_score', ascending=False)

In [ ]:
from river.tree import SGTRegressor

# model pipeline
model = SGTRegressor()

# fit the model
for x_i,y_i in zip(X_train,y_train):
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    model.learn_one(x_json,y_i)

# predict on the test set
preds = []
for x_i in X_test:
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    preds.append(model.predict_one(x_json))

# compute accuracy
r2_score(y_test, preds)


In [ ]:
from river.tree import SGTRegressor

def evaluate_SGT(delta, lambda_value, grace_period):

    # model pipeline 
    model = SGTRegressor(delta=delta,
                        lambda_value=lambda_value,
                        grace_period=grace_period,)

    # fit the model
    for x_i,y_i in zip(X_train,y_train):
        x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
        model.learn_one(x_json,y_i)

    # predict on the test set
    preds = []
    for x_i in X_test:
        x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
        preds.append(model.predict_one(x_json))

    # compute accuracy
    return r2_score(y_test, preds)


In [ ]:
grace_periods=[0,10,25]
lambda_values=[0.5, 0.8, 1.]
deltas=[0.0001, 0.001, 0.01, 0.1]


In [ ]:
results = []
i = 0
for grace_period in grace_periods:
    for lambda_value in lambda_values:
        for delta in deltas:
            print(i)
            i = i+1
            result = evaluate_SGT(delta, lambda_value, grace_period)
            print(result)
            results.append([delta, lambda_value, grace_period,result])


In [ ]:
pd.DataFrame(results, columns=['delta', 'lambda_value', 'grace_period', 'r2_score' ]).sort_values('r2_score', ascending=False)

In [ ]:
from river.ensemble import SRPRegressor

# model pipeline 
base_model = LinearRegression()
model = SRPRegressor(
    model=base_model,
    n_models=3,
    seed=42
)

# fit the model
for x_i,y_i in zip(X_train,y_train):
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    model.learn_one(x_json,y_i)

# predict on the test set
preds = []
for x_i in X_test:
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    preds.append(model.predict_one(x_json))

# compute accuracy
r2_score(y_test, preds)


In [ ]:
def evaluate_SRP(n_models):
    # model pipeline 
    base_model = LinearRegression()
    model = SRPRegressor(
        model=base_model,
        n_models=n_models,
        seed=42
    )

    # fit the model
    for x_i,y_i in zip(X_train,y_train):
        x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
        model.learn_one(x_json,y_i)

    # predict on the test set
    preds = []
    for x_i in X_test:
        x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
        preds.append(model.predict_one(x_json))

    # compute accuracy
    return r2_score(y_test, preds)


In [ ]:
results = []
for n_models in range(1, 50):
    results.append([n_models, evaluate_SRP(n_models)])


In [ ]:
pd.DataFrame(results,columns=['n_models', 'r2_score']).sort_values('r2_score', ascending=False)